### 1. Importar librerías para abrir coordenadas en archivos de polígonos .shx y .shp:

In [98]:
import geopandas as gpd                                             # leer los archivos .shx y .shp
from shapely.geometry import Polygon, MultiPolygon, shape           # Simplificar Polígonos
from shapely.ops import unary_union                                 # Combinar polígonos

# Cargar los datos geoespaciales
data = gpd.read_file('../Datasets/taxi_zones.shx')


para poder leer el archivo taxi_zones.shx en la misma carpeta debe estar el arcivo taxi_zones.shp

### 2. Transformar coordenadas planas ATEST WELL-KNOWN IDENTIFIER 2263 a la proyección inversa mundial (EPSG:4326):

In [99]:
gdf = gpd.GeoDataFrame(data)

# Definir EPSG:2263 que está en pies (LATEST WELL-KNOWN IDENTIFIER 2263):
gdf.crs = 'EPSG:2263'

# Corregir coordenadas planas a mundiales:
gdf_latlon = gdf.to_crs('EPSG:4326')  # Proyección inversa al sistema de coordenadas geográficas

# Calcular los centroides de los polígonos
gdf_latlon['centroid'] = gdf_latlon['geometry'].centroid

# Extraer las coordenadas x e y de los centroides
gdf_latlon['centroid_x'] = gdf_latlon['centroid'].x
gdf_latlon['centroid_y'] = gdf_latlon['centroid'].y

C:\Users\davin\AppData\Local\Temp\ipykernel_15384\804222979.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_latlon['centroid'] = gdf_latlon['geometry'].centroid


### 3. Ordenar y selecionar columnas:

In [100]:
# Seleccionar las columnas deseadas
gdf_latlon = gdf_latlon[['OBJECTID', 'borough', 'zone', 'centroid_x', 'centroid_y','geometry']]

# Renombrar las columnas centroid_x y centroid_y a longitud y latitud respectivamente
gdf_latlon = gdf_latlon.rename(columns={'OBJECTID':'LocationID','centroid_x': 'longitud', 'centroid_y': 'latitud'})

### 4. Simplificar polígonos:

In [101]:
# Definir la tolerancia para la simplificación
tolerancia = 0.0001  # ajusta este valor según sea necesario

# Función para simplificar una geometría individual
def simplificar_geometria(geom, tolerancia):
    return geom.simplify(tolerancia)

# Aplicar la función de simplificación a cada geometría en el GeoDataFrame
gdf_latlon['geometry'] = gdf_latlon['geometry'].apply(lambda geom: simplificar_geometria(geom, tolerancia))

### 5. Eliminar datos duplicados unificando los polígonos de los ID 55+56 y 103+104+105:

In [102]:
# Seleccionar los polígonos correspondientes a los LocationID 56 y 57
poligonos_56_57 = gdf_latlon.loc[gdf_latlon['LocationID'].isin([56, 57]), 'geometry']

# Crear un MultiPolygon combinando los polígonos
multi_poligono_56_57 = unary_union(poligonos_56_57)

# Reemplazar el polígono del LocationID 56 con el MultiPolygon combinado
gdf_latlon.loc[gdf_latlon['LocationID'] == 56, 'geometry'] = multi_poligono_56_57

# Eliminar el registro correspondiente al LocationID 57
gdf_latlon= gdf_latlon[gdf_latlon['LocationID'] != 57]

In [103]:
# Seleccionar los polígonos correspondientes a los LocationID 103, 104 y 105
poligonos_103_104_105 = gdf_latlon.loc[gdf_latlon['LocationID'].isin([103, 104, 105]), 'geometry']

# Crear un MultiPolygon combinando los polígonos
multi_poligono_103_104_105 = unary_union(poligonos_103_104_105)

# Reemplazar el polígono del LocationID 103 con el MultiPolygon combinado
gdf_latlon.loc[gdf_latlon['LocationID'] == 103, 'geometry'] = multi_poligono_103_104_105

# Eliminar el registro correspondiente al LocationID 104 y 105
gdf_latlon= gdf_latlon[gdf_latlon['LocationID'] != 104]
gdf_latlon= gdf_latlon[gdf_latlon['LocationID'] != 105]

Se guarda archivo como Taxi_Zone.csv en DataClean